# Experiment-Analyse: Multi-Agent System

Dieses Notebook analysiert die Ergebnisse der CrewAI Multi-Agent Experimente.

## Inhalt
1. Daten laden
2. Übersicht aller Experimente
3. Visualisierungen (Diagramme)
4. Statistische Auswertung
5. Export für wissenschaftliche Arbeit

## 1. Setup und Daten laden

In [ ]:
# Bibliotheken importieren
import json
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Matplotlib Stil
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("✅ Bibliotheken geladen!")

In [ ]:
# Experiment-Daten laden
# Sucht zuerst in "projekte", dann in "experiments" für Rückwärtskompatibilität
EXPERIMENTS_DIR = Path("projekte")
if not EXPERIMENTS_DIR.exists():
    EXPERIMENTS_DIR = Path("experiments")
    print("⚠️ 'projekte' Ordner nicht gefunden, verwende 'experiments'")
else:
    print(f"📁 Verwende Ordner: {EXPERIMENTS_DIR}")

def load_experiment(exp_dir):
    """Lädt ein einzelnes Experiment aus dem Ordner."""
    json_files = list(exp_dir.glob("*_full.json"))
    if not json_files:
        return None
    
    with open(json_files[0], "r", encoding="utf-8") as f:
        data = json.load(f)
    
    return {
        "id": data["config"]["experiment_id"],
        "name": data["config"]["experiment_name"],
        "timestamp": data["config"]["timestamp"],
        "models": data["config"]["models"],

## 2. Übersichtstabelle

In [ ]:
# DataFrame erstellen für Übersicht
overview_data = []
for exp in experiments:
    overview_data.append({
        "Experiment": exp["name"],
        "Timestamp": exp["timestamp"][:19],
        "Developer Model": exp["models"].get("developer", "N/A"),
        "Dauer (s)": exp["duration"],
        "Tokens": exp["tokens"],
        "Erfolgreich": "Ja" if exp["success"] else "Nein"
    })

df_overview = pd.DataFrame(overview_data)
df_overview

## 3. Visualisierungen

In [ ]:
# Balkendiagramm: Dauer pro Experiment
fig, ax = plt.subplots(figsize=(10, 5))

names = [exp["name"] for exp in experiments]
durations = [exp["duration"] for exp in experiments]
colors = ["#2ecc71" if exp["success"] else "#e74c3c" for exp in experiments]

bars = ax.bar(names, durations, color=colors, edgecolor="black")
ax.set_xlabel("Experiment")
ax.set_ylabel("Dauer (Sekunden)")
ax.set_title("Ausführungszeit pro Experiment")
plt.xticks(rotation=45, ha="right")

# Werte auf Balken anzeigen
for bar, duration in zip(bars, durations):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
            f"{duration:.1f}s", ha="center", fontsize=10)

plt.tight_layout()
plt.savefig("charts/dauer_pro_experiment.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# Charts-Ordner erstellen falls nicht vorhanden
Path("charts").mkdir(exist_ok=True)

# Balkendiagramm: Tokens pro Experiment
fig, ax = plt.subplots(figsize=(10, 5))

tokens = [exp["tokens"] for exp in experiments]

bars = ax.bar(names, tokens, color="#3498db", edgecolor="black")
ax.set_xlabel("Experiment")
ax.set_ylabel("Geschätzte Tokens")
ax.set_title("Token-Nutzung pro Experiment")
plt.xticks(rotation=45, ha="right")

for bar, token in zip(bars, tokens):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
            f"{token}", ha="center", fontsize=10)

plt.tight_layout()
plt.savefig("charts/tokens_pro_experiment.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# Dauer pro Agent (gemittelt über alle Experimente)
agent_durations = {}
agent_tokens = {}

for exp in experiments:
    for metric in exp["agent_metrics"]:
        role = metric["agent_role"]
        if role not in agent_durations:
            agent_durations[role] = []
            agent_tokens[role] = []
        agent_durations[role].append(metric["duration_seconds"])
        agent_tokens[role].append(metric["estimated_output_tokens"])

# Durchschnitte berechnen
avg_durations = {role: sum(vals)/len(vals) for role, vals in agent_durations.items()}
avg_tokens = {role: sum(vals)/len(vals) for role, vals in agent_tokens.items()}

# Visualisierung
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Dauer pro Agent
colors = ["#2ecc71", "#3498db", "#e74c3c", "#9b59b6"]
roles = list(avg_durations.keys())
durations = list(avg_durations.values())

axes[0].bar(roles, durations, color=colors[:len(roles)], edgecolor="black")
axes[0].set_xlabel("Agent")
axes[0].set_ylabel("Ø Dauer (Sekunden)")
axes[0].set_title("Durchschnittliche Dauer pro Agent")
axes[0].tick_params(axis='x', rotation=45)

# Tokens pro Agent
tokens = list(avg_tokens.values())
axes[1].bar(roles, tokens, color=colors[:len(roles)], edgecolor="black")
axes[1].set_xlabel("Agent")
axes[1].set_ylabel("Ø Tokens")
axes[1].set_title("Durchschnittliche Token-Nutzung pro Agent")
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig("charts/agent_vergleich.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# Kreisdiagramm: Anteil der Zeit pro Agent
fig, ax = plt.subplots(figsize=(8, 8))

total_duration = sum(avg_durations.values())
percentages = [d/total_duration*100 for d in avg_durations.values()]

wedges, texts, autotexts = ax.pie(
    percentages, 
    labels=roles, 
    autopct="%1.1f%%",
    colors=colors[:len(roles)],
    explode=[0.02]*len(roles),
    shadow=True
)
ax.set_title("Zeitverteilung nach Agent")

plt.tight_layout()
plt.savefig("charts/zeitverteilung_pie.png", dpi=150, bbox_inches="tight")
plt.show()

## 4. Statistische Auswertung

In [ ]:
# Agent-Metriken als DataFrame
all_metrics = []
for exp in experiments:
    for metric in exp["agent_metrics"]:
        all_metrics.append({
            "Experiment": exp["name"],
            "Agent": metric["agent_role"],
            "Modell": metric["model"],
            "Dauer (s)": metric["duration_seconds"],
            "Output Tokens": metric["estimated_output_tokens"],
            "Erfolg": metric["success"]
        })

df_metrics = pd.DataFrame(all_metrics)
df_metrics

In [ ]:
# Statistiken pro Agent
stats = df_metrics.groupby("Agent").agg({
    "Dauer (s)": ["mean", "std", "min", "max"],
    "Output Tokens": ["mean", "std", "min", "max"]
}).round(2)

print("📊 Statistische Auswertung pro Agent:\n")
stats

In [ ]:
# Gesamtstatistiken
print("📊 Gesamtstatistiken:\n")
print(f"Anzahl Experimente: {len(experiments)}")
print(f"Erfolgreiche Experimente: {sum(1 for e in experiments if e['success'])}")
print(f"")
print(f"Gesamtdauer:")
print(f"  - Mittelwert: {df_overview['Dauer (s)'].mean():.2f} Sekunden")
print(f"  - Minimum: {df_overview['Dauer (s)'].min():.2f} Sekunden")
print(f"  - Maximum: {df_overview['Dauer (s)'].max():.2f} Sekunden")
print(f"")
print(f"Token-Nutzung:")
print(f"  - Mittelwert: {df_overview['Tokens'].mean():.0f} Tokens")
print(f"  - Minimum: {df_overview['Tokens'].min()} Tokens")
print(f"  - Maximum: {df_overview['Tokens'].max()} Tokens")

## 5. Export für wissenschaftliche Arbeit

In [ ]:
# CSV Export
df_overview.to_csv("export_experimente.csv", index=False, encoding="utf-8")
df_metrics.to_csv("export_agent_metriken.csv", index=False, encoding="utf-8")
print("✅ CSV-Dateien exportiert:")
print("   - export_experimente.csv")
print("   - export_agent_metriken.csv")

In [ ]:
# LaTeX Tabelle generieren
print("📄 LaTeX-Tabelle für wissenschaftliche Arbeit:\n")
print(r"\begin{table}[h]")
print(r"\centering")
print(r"\caption{Übersicht der durchgeführten Experimente}")
print(r"\begin{tabular}{|l|c|c|c|}")
print(r"\hline")
print(r"Experiment & Modell & Dauer (s) & Tokens \\")
print(r"\hline")
for exp in experiments:
    model = exp["models"].get("developer", "N/A")
    print(f"{exp['name']} & {model} & {exp['duration']:.1f} & {exp['tokens']} \\\\")
print(r"\hline")
print(r"\end{tabular}")
print(r"\end{table}")

In [ ]:
# Systeminfo anzeigen (für Reproduzierbarkeit)
if experiments:
    sys_info = experiments[0].get("system_info", {})
    print("🖥️ Systemkonfiguration (für Reproduzierbarkeit):\n")
    print(f"  Platform: {sys_info.get('platform', 'N/A')}")
    print(f"  Python: {sys_info.get('python_version', 'N/A')}")
    print(f"  CPU Cores: {sys_info.get('cpu_count', 'N/A')}")
    print(f"  RAM Total: {sys_info.get('ram_total_gb', 'N/A')} GB")

## 📁 Generierte Dateien

Nach Ausführung dieses Notebooks findest du:

| Datei | Beschreibung |
|-------|-------------|
| `charts/dauer_pro_experiment.png` | Balkendiagramm: Ausführungszeit |
| `charts/tokens_pro_experiment.png` | Balkendiagramm: Token-Nutzung |
| `charts/agent_vergleich.png` | Vergleich der Agenten |
| `charts/zeitverteilung_pie.png` | Kreisdiagramm: Zeitanteile |
| `export_experimente.csv` | Übersicht für Excel/SPSS |
| `export_agent_metriken.csv` | Detaillierte Metriken |